In [2]:
# Add python 2 and python 2 support
from __future__ import division, print_function, unicode_literals

In [3]:
# Import libraries
import os
import tarfile
import urllib
import numpy as np
import numpy.random as rnd
import pandas as pd

# Random seed
rnd.seed(7)

# Plotting libraries
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt